In [21]:
!pip install tiktoken --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.5 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount("/data")

Mounted at /data


In [3]:
cd /data/MyDrive/Dataset

/data/MyDrive/Dataset


In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tiktoken

In [23]:
def count_tokens(text):
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
    return len(encoding.encode(text))

In [7]:
raw_data_part1 = pd.read_csv("data/raw_data_part1.csv")
raw_data_part1.head()

,Text,WallId,PostId,Source,GptResponse
0,б/у:\n∙ сумка дорожная на колесах (~высота 70с...,1332882905,570967,tg,"{\n ""place"": null,\n ""bundles"": [\n {\n ..."
1,davines - мусс-объем \nthis is a volume boosti...,1332882905,618057,tg,"{\n ""place"": ""вера"",\n ""bundles"": [\n {\n..."
2,👌Новая поступления 🥳🥳🥳\n. tencel \n Ц...,-192083416,13589,vk,"{\n ""place"": ""https://vk.com/wall488363770_21..."
3,"клининг/фея чистоты/ уборка 🧚\nофис, квартира,...",1121942433,400700,tg,"{\n ""place"": ""по маршруту автобусов 711, 424,..."
4,Распродажа \nЦена: 550р шт \nРазмер: 41.42.43....,-206817857,82815,vk,"{\n ""place"": null,\n ""bundles"": []\n}"


In [12]:
raw_data_part2 = pd.read_csv("data/labeled_data_part2.csv").drop(columns="Unnamed: 0")
raw_data_part2.head()

,Text,WallId,PostId,Source,GptResponse
0,продам монитор dell 22'' - модель se2216h / fu...,1332882905,846910,tg,"{\n ""place"": ""район метро надзаладеви/гоцирид..."
1,playstation ps4 на 1тб с 2-мая геймпадами и за...,1121942433,664671,tg,"{\n ""place"": ""лимассол, район папаса"",\n ""bu..."
2,#БНИ_ПродамБУ_Саяногорск\n#БНИ_ПродамБУ_Почта\...,-114967596,573555,vk,"{\n ""place"": ""Саяногорск, Почта, СДЭК, Авито""..."
3,Прогулочная коляска Be2Me\nВ отличном состояни...,-82544098,641181,vk,"{\n ""place"": null,\n ""bundles"": [\n {\n ..."
4,iphone 13 pro maxy 2550 lari 512 gb❤️‍🔥👍,1332882905,746806,tg,"{\n ""place"": """",\n ""bundles"": [\n {\n ..."


In [15]:
raw_data_part3 = pd.read_csv("data/raw_data_part3.csv").drop(columns="Unnamed: 0")
raw_data_part3.head()

,Text,WallId,PostId,Source,Title,Price,Currency,Count,Label_gpt,Comment_gpt
0,Распродажа \nКоллагеновый консилер от Enough\n...,-206817857,86686,vk,Коллагеновый консилер от Enough,80,RUB,1,valid,"The offer clearly specifies the product, count..."
1,#продаю в тбилиси focusrite scatlett 2i2. полн...,1332882905,439179,tg,focusrite scatlett 2i2,190,₾,1,valid,The offer contains all necessary information a...
2,"Ботильоны Vagabond, натуральная кожа/мех, 39 р...",-48332904,713045,vk,"Ботильоны Vagabond, натуральная кожа/мех, разм...",5000,RUB,1,valid,The offer clearly describes a single bundle of...
3,"Цена: 700р\nраспродажа \nРазмеры: 42,44,46,48,...",-206817857,120264,vk,флисовая одежда,700,RUB,1,invalid,The offer lacks specific details about the bun...
4,"продам кухонный стол. стол удобный, основатель...",1121942433,699812,tg,кухонный стол и 4 стула,150,EUR,1,valid,The offer clearly specifies that it includes a...


In [16]:
raw_data_part1.shape, raw_data_part2.shape, raw_data_part3.shape

((5001, 5), (5001, 5), (5153, 10))

In [24]:
data = pd.concat([raw_data_part1, raw_data_part2])
data.duplicated().sum()

0

In [49]:
model = "gpt-4-turbo-2024-04-09"
input_price = 10 / 1000000
output_price = 30 / 1000000

label_prompt = """You are an AI trainer at a marketplace company. Your role is to convert product offer posts from plain text into structured JSON format. A "bundle" is defined as a set of items being sold together, which cannot be separated.
The JSON structure must include:
"place": The location where the buyer can pick up the product or delivery information, if available.
"bundles": A list of dictionaries, each representing a valid bundle.
Each bundle dictionary should contain ONLY the following information for ONE bundle:
"name" (string): A brief title of the bundle, including enough details for the customer to understand what is being sold. Ensure to include important general details, such as product category, if provided.
"quantity" (string): The minimal unseparable amount of product in the bundle. For countable items, this is the number of items. For non-countable products (like liquids or materials), this should reflect the volume, length, or other relevant metrics (default is '1').
"price" (number): The cost for the entire bundle, based on the 'quantity' specified.
"currency" (string): The currency for the 'price' (default is 'RUB').

Regarding product variations, such as size or color, these should NOT be treated as separate bundles. Variations should be considered additional descriptions and are not required in the JSON. The focus is on distinct products or bundles as defined by their essential characteristics, not their variations.
Exclude any fields with null values from the JSON. Do not add descriptions or any fields other than those specified. If no price information is provided, exclude the bundle from the JSON. Answer in the language of the original offer.
Your response should be a JSON only, with no additional text required.
"""

In [50]:
data["InputTokens"] = count_tokens(label_prompt) + data["Text"].apply(count_tokens)
data["OutputTokens"] = data["GptResponse"].apply(count_tokens)

# Умножаем на 1,5, потому что часть сырых данных (которая была преобразована в raw_data_part3) не сохранилась,
# но датасет был перемешан, и ожидаемая стоимость разметки 3-й части приблизительно равна стоимости разметки первой или второй части
label_price = (data["InputTokens"].sum() * input_price + data["OutputTokens"].sum() * output_price) * 1.5

f"Labeling of the dataset costs {np.round(label_price)} USD"

'Labeling of the dataset costs 106.0 USD'

In [51]:
final_data = pd.read_csv("data/labeled_15k.csv").drop(columns="Unnamed: 0")
print(final_data.shape)
final_data.head()

(19214, 10)


,Text,WallId,PostId,Source,Title,Price,Currency,Count,Label_gpt,Comment_gpt
0,"1. монитор, 250 лари\n2. мольберт, 50 лари\n3....",1332882905,686748,tg,"рюкзак, 40 литров, идеальное состояние",100,лари,1,valid,The offer includes a backpack bundle that matc...
1,#БНИ_ПродамБУ_Москва\n#БНИ_ПродамНовая_Москва\...,-114967596,592027,vk,"Street Fighter (база, стретчи, доп)",26000,RUB,1,valid,The offer clearly describes the bundle's conte...
2,Продам стульчик для кормления Happy baby. В хо...,-82544098,661957,vk,Стульчик для кормления Happy baby,3000,RUB,1,valid,The offer clearly describes a single feeding c...
3,"Продаю компьютер Цена 55000 Все почти новое ,е...",-90442795,353383,vk,Компьютер с комплектующими и аксессуарами,55000,RUB,1,valid,The offer clearly describes a computer with co...
4,Новинка сезона \nКачество очень хорошее \nЦена...,-192083416,15466,vk,Новинка сезона,2000,RUB,1,invalid,"The OFFER lacks detailed product information, ..."


In [52]:
val_prompt = """You are a quality control officer at a marketplace. You have received a product offer from a seller and a product description in JSON format. A "bundle" is defined as an inseparable set of items being sold together, with a price provided for the entire set. Prices in the offer should not be divided.

Your task is to determine whether the offer includes the described bundle. You also need to handle the specified case, where the "OFFER" lacks sufficient detail about the product and makes it impossible for the customer to understand or verify the product without additional context and set "invalid" response.

The expected JSON fields are:
- **"Title" (string)**: A brief title of the bundle, including information about what is being sold. Important general details, such as product category, should be included, if provided. The title must give customers enough information to understand what the bundle contains without any external context or needing to perform additional research.
- **"Count" (string)**: The minimal inseparable amount of product in the bundle. For countable items, this is the number of items. For non-countable products (like liquids or materials), this should reflect the volume, length, or other relevant metrics (default is '1').
- **"Price" (number)**: The cost for the entire bundle, based on the 'Count' specified.
- **"Currency" (string)**: The currency for the 'Price' (default is 'RUB').

The JSON product description should represent just ONE bundle.

Bundles must not be split up, and separate products must not be aggregated into a single bundle.

Respond with JSON with ONLY 2 fields:
- **"Label"**: should be one of ("valid", "invalid"):
    - 'valid' if the product represented by the bundle is valid, and the OFFER contains the described bundle, meaning it is possible to purchase the described bundle in the 'Count' for the 'Price' specified in 'Currency'.
    - 'invalid' - if the OFFER does not provide enough information for a customer to understand what the product bundle contains without additional context, or if the product described seems unreal or cannot be verified based on the information provided.
- **"Comment"**: Explain your decision in one sentence.

Note: This instruction emphasizes the need for clarity and verifiability in the product title and details, ensuring that customers can understand and verify what they are purchasing without needing to search for more information.

Answer with one JSON only, no extra information should be provided."""

def product_desc(row):
    s = f'''
    OFFER: {row.Text}

    Bundle description
    Title: {row.Title}
    Count: {row.Count}
    Price: {row.Price}
    Currency: {row.Currency}
    '''
    return s

def get_response(row):
    return str({"Label": row.Label_gpt, "Comment": row.Comment_gpt})

In [53]:
final_data["Description"] = final_data.apply(product_desc, axis=1)
final_data["GptResponse"] = final_data.apply(get_response, axis=1)
final_data.head(2)

,Text,WallId,PostId,Source,Title,Price,Currency,Count,Label_gpt,Comment_gpt,Description,GptResponse
0,"1. монитор, 250 лари\n2. мольберт, 50 лари\n3....",1332882905,686748,tg,"рюкзак, 40 литров, идеальное состояние",100,лари,1,valid,The offer includes a backpack bundle that matc...,"\n OFFER: 1. монитор, 250 лари\n2. мольберт...","{'Label': 'valid', 'Comment': 'The offer inclu..."
1,#БНИ_ПродамБУ_Москва\n#БНИ_ПродамНовая_Москва\...,-114967596,592027,vk,"Street Fighter (база, стретчи, доп)",26000,RUB,1,valid,The offer clearly describes the bundle's conte...,\n OFFER: #БНИ_ПродамБУ_Москва\n#БНИ_Продам...,"{'Label': 'valid', 'Comment': ""The offer clear..."


In [54]:
final_data["InputTokens"] = count_tokens(val_prompt) + final_data["Description"].apply(count_tokens)
final_data["OutputTokens"] = final_data["GptResponse"].apply(count_tokens)

val_price = (final_data["InputTokens"].sum() * input_price + final_data["OutputTokens"].sum() * output_price)

f"Validation of the dataset costs {np.round(val_price)} USD"

'Validation of the dataset costs 157.0 USD'